# Algorithm Cage Match

This notebook generates sets of fake STTCs from PTCs, and determines the mean square error produced by TEPs-I, Bagheri and my count matching algorithms.

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd
from traffic_prophet import cfg
import pathlib, os
import configparser

from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import growthfactor as gf
from traffic_prophet.countmatch import neighbour

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

filepath = pathlib.Path.home().joinpath('.charlesconfig')
if os.path.isfile(filepath):
    vol_conn = connection.Connection(filepath, 'POSTGRES',
                                     'czhu.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'POSTGRES',
                                    'czhu.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['MAPBOX']['token']
    PLOTLY_USER = config['PLOTLY']['user']
    PLOTLY_KEY = config['PLOTLY']['key']
else:
    filepath = pathlib.Path.home().joinpath('cf.txt')
    vol_conn = connection.Connection(filepath, 'localpg',
                                     'prj_vol.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'localpg',
                                    'gis.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['mapbox']['token']
    PLOTLY_USER = config['plotly']['user']
    PLOTLY_KEY = config['plotly']['apikey']

In [2]:
rdr = reader.Reader(vol_conn)
%time rdr.read()

CPU times: user 1min 36s, sys: 2.09 s, total: 1min 38s
Wall time: 1min 49s


In [3]:
gf.get_growth_factors(rdr)

In [4]:
ptc_ids = np.unique(np.abs(list(rdr.ptcs.keys())))
nb = neighbour.NeighbourLonLatEuclidean(ll_conn, 20, ptc_ids)
%time nb.find_neighbours()

CPU times: user 18.2 s, sys: 531 ms, total: 18.8 s
Wall time: 18.5 s


## STTC-MC generator

This routine randomly generates a set of STTCs 

In [5]:
p = rdr.ptcs[-30022069]

In [6]:
doyr = p.data['Daily Count'].loc[:, ['Date']].copy()

In [7]:
    year = 2010    
    doyr.loc[year, 'Day-to-AADT Ratio'] = (
            p.data['AADT'].at[year, 'AADT'] /
            p.data['Daily Count'].loc[year, 'Daily Count']).values

In [8]:
doyr

Date  Day-to-AADT Ratio
Year Day of Year                              
2010 1           2010-01-01           1.439357
     2           2010-01-02           1.098346
     3           2010-01-03           1.284496
     6           2010-01-06           0.952754
     7           2010-01-07           0.939115
     8           2010-01-08           0.896251
     9           2010-01-09           0.938695
     10          2010-01-10           1.058197
     11          2010-01-11           0.926145
     12          2010-01-12           0.896869
     16          2010-01-16           0.895644
     17          2010-01-17           1.046237
     18          2010-01-18           0.946408
     19          2010-01-19           0.896517
     20          2010-01-20           0.899928
     21          2010-01-21           0.860825
     22          2010-01-22           0.818632
     23          2010-01-23           0.877987
     24          2010-01-24           1.060220
     25          2010-01-25           0.937889
     26          2010-01-26           0.875744
     27          2010-01-27           0.879819
     28          2010-01-28           0.854778
     29          2010-01-29           0.825258
     30          2010-01-30           0.932706
     32          2010-02-01           0.915146
     33          2010-02-02           0.896400
     34          2010-02-03           0.897690
     35          2010-02-04           0.853772
     38          2010-02-07           1.061159
...                     ...                ...
2017 326         2017-11-22                NaN
     329         2017-11-25                NaN
     330         2017-11-26                NaN
     331         2017-11-27                NaN
     332         2017-11-28                NaN
     333         2017-11-29                NaN
     334         2017-11-30                NaN
     335         2017-12-01                NaN
     336         2017-12-02                NaN
     337         2017-12-03                NaN
     338         2017-12-04                NaN
     339         2017-12-05                NaN
     340         2017-12-06                NaN
     341         2017-12-07                NaN
     342         2017-12-08                NaN
     343         2017-12-09                NaN
     345         2017-12-11                NaN
     346         2017-12-12                NaN
     347         2017-12-13                NaN
     349         2017-12-15                NaN
     350         2017-12-16                NaN
     351         2017-12-17                NaN
     352         2017-12-18                NaN
     353         2017-12-19                NaN
     355         2017-12-21                NaN
     356         2017-12-22                NaN
     357         2017-12-23                NaN
     358         2017-12-24                NaN
     359         2017-12-25                NaN
     363         2017-12-29                NaN

[878 rows x 2 columns]

In [79]:
import importlib

In [80]:
import countmatch_teps as cmt
importlib.reload(cmt)

<module 'countmatch_teps' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_teps.py'>